In [ ]:
# Running a Random Forest as Baseline and for Feature Selection 

In [ ]:
# Data: MoH_Vote_Data_Long_Sorted.csv 
# Contains Donation Data by Disney and the rations of voting along party line in 5 categories over 4 sessios

In [2]:
import csv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [18]:
# Get Data
x = "~/Documents/GitHub/CampaignDonations/Output_Data/MoH_Vote_Data_Long_Sorted.csv"
df = pd.read_csv(x)
df.head()

,level_0,index,person,Year,wikidataid,sex,last,person_y,FirstName,first,...,religion,birthplace,disney_donations,VoteAlongRatio_Film,VoteAlongRatio_FS_IIR,VoteAlongRatio_FS_FSS,VoteAlongRatio_FS_CM,VoteAlongRatio_FS_REB,disney_donations_lagMinus1,disney_donations_lagPlus1
0,0,0,4,2012,Q748066,male,Blumenauer,400033.0,Earl,Earl,...,0,"45.51667,-122.66667",0.0,0.000000,0.333333,0.333333,0.571429,0.666667,0.0,NaN
1,229,229,4,2014,Q748066,male,Blumenauer,400033.0,Earl,Earl,...,0,"45.51667,-122.66667",0.0,0.333333,0.800000,0.333333,0.400000,0.500000,0.0,0.0
2,458,458,4,2016,Q748066,male,Blumenauer,400033.0,Earl,Earl,...,0,"45.51667,-122.66667",0.0,0.400000,0.428571,0.222222,0.000000,0.000000,1000.0,0.0
3,687,687,4,2018,Q748066,male,Blumenauer,400033.0,Earl,Earl,...,0,"45.51667,-122.66667",1000.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,0.0
4,1,1,10,2012,Q1289889,male,Clyburn,400075.0,James,James,...,protestantism methodist,"33.92056,-80.34472",0.0,0.500000,0.500000,0.666667,0.571429,0.666667,2000.0,NaN


In [19]:
#df.columns.tolist()

In [20]:
# Clean Data
list = ["birthplace",'level_0',
 'index','wikidataid','last','person_y','FirstName','first',
 'death',
 'Number_Sessions_Served',
 'person_x',
 'party_x',
 'Session_114',
 'name',
 'birth',
 'party_y',
 'deathplace',
 'Session_116',
 'religion',    
 'ID_NameParty',
 'Session_113',
 'LastName',
 'pageid',
 'Session_115',
 'wikititle',
 'Pure_name',
 'name_y',
 'birthplace']

df = df.drop(columns = list)

# Get dummines for "character"-data

df = pd.get_dummies(df, columns = ['sex', 'ethnicity', "party_short"])
df.head()


,person,Year,disney_donations,VoteAlongRatio_Film,VoteAlongRatio_FS_IIR,VoteAlongRatio_FS_FSS,VoteAlongRatio_FS_CM,VoteAlongRatio_FS_REB,disney_donations_lagMinus1,disney_donations_lagPlus1,sex_female,sex_male,ethnicity_asian,ethnicity_black,ethnicity_hispanic,ethnicity_islander,ethnicity_native,ethnicity_white,party_short_D,party_short_R
0,4,2012,0.0,0.000000,0.333333,0.333333,0.571429,0.666667,0.0,NaN,0,1,0,0,0,0,0,1,1,0
1,4,2014,0.0,0.333333,0.800000,0.333333,0.400000,0.500000,0.0,0.0,0,1,0,0,0,0,0,1,1,0
2,4,2016,0.0,0.400000,0.428571,0.222222,0.000000,0.000000,1000.0,0.0,0,1,0,0,0,0,0,1,1,0
3,4,2018,1000.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,0.0,0,1,0,0,0,0,0,1,1,0
4,10,2012,0.0,0.500000,0.500000,0.666667,0.571429,0.666667,2000.0,NaN,0,1,0,1,0,0,0,0,1,0


In [23]:
# Split Data into Train and Test Sets

#create a list of all unique IDs
ids=df
data_id=ids.drop_duplicates(subset='person')
data_id=data_id[['person']]
data_id.shape

,person
0,4
4,10
8,13
12,15
16,16


In [24]:
# Create new variable with random number between one and zero

np.random.seed(42)
data_id['data_use'] = (np.random.randint(0, 10000, data_id.shape[0]))/10000
data_id=data_id[['person', 'data_use']]
data_id.head()


,person,data_use
0,4,0.7270
4,10,0.0860
8,13,0.5390
12,15,0.5191
16,16,0.5734


In [25]:
# Assign each person to a modeling group training or testing 

data_id['MODELING_GROUP'] = np.where(((data_id.data_use <= 0.55)), 'TRAINING', 'TESTING')
data_id.head()

,person,data_use,MODELING_GROUP
0,4,0.7270,TESTING
4,10,0.0860,TRAINING
8,13,0.5390,TRAINING
12,15,0.5191,TRAINING
16,16,0.5734,TESTING


In [26]:
groups = data_id.groupby(['MODELING_GROUP'])['person'].count()
groups

MODELING_GROUP
TESTING     110
TRAINING    119
Name: person, dtype: int64

In [27]:

df=df.sort_values(by=['person'], ascending=[True])
data_id=data_id.sort_values(by=['person'], ascending=[True])
df_modelgroups =df.merge(data_id, on=['person'], how='inner')
df_modelgroups.head()

,person,Year,disney_donations,VoteAlongRatio_Film,VoteAlongRatio_FS_IIR,VoteAlongRatio_FS_FSS,VoteAlongRatio_FS_CM,VoteAlongRatio_FS_REB,disney_donations_lagMinus1,disney_donations_lagPlus1,...,ethnicity_asian,ethnicity_black,ethnicity_hispanic,ethnicity_islander,ethnicity_native,ethnicity_white,party_short_D,party_short_R,data_use,MODELING_GROUP
0,4,2012,0.0,0.000000,0.333333,0.333333,0.571429,0.666667,0.0,NaN,...,0,0,0,0,0,1,1,0,0.727,TESTING
1,4,2014,0.0,0.333333,0.800000,0.333333,0.400000,0.500000,0.0,0.0,...,0,0,0,0,0,1,1,0,0.727,TESTING
2,4,2016,0.0,0.400000,0.428571,0.222222,0.000000,0.000000,1000.0,0.0,...,0,0,0,0,0,1,1,0,0.727,TESTING
3,4,2018,1000.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,0.0,...,0,0,0,0,0,1,1,0,0.727,TESTING
4,10,2012,0.0,0.500000,0.500000,0.666667,0.571429,0.666667,2000.0,NaN,...,0,1,0,0,0,0,1,0,0.086,TRAINING


In [ ]:
# Now start that random forrest shit....